https://www.analyticsvidhya.com/blog/2019/08/11-important-model-evaluation-error-metrics/

In [95]:
#this is needed for correct module importing
%cd /Users/paolobonomi/Work/Python/CapsNetwork
import sys
sys.path.append("/Users/paolobonomi/Work/Python/CapsNetwork/src")

import numpy as np
import pandas
from model_10_epochs import model, testing, dataset, training_dataset_size, testing_dataset_size
from capsuleNetwork import CapsuleNetwork

/Users/paolobonomi/Work/Python/CapsNetwork


In [96]:
headers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [90]:
def getAccuracy(model, data, data_size):
    test_sum = 0
    for X_batch, y_batch in data:
        test_sum += sum(CapsuleNetwork.predict(model, X_batch)==y_batch.numpy())
    return test_sum/data_size

def getConfusionMatrix(model, data):
    matrix = np.zeros((10,10))

    for X_batch, y_batch in data:
        res = CapsuleNetwork.predict(model, X_batch)
        for i in range(0, y_batch.shape[0]):
            matrix[y_batch[i], res[i]] += 1

    return matrix

def normalize_matrix(mat):
    for i in range(0, 10):
        n = 0
        for j in range(0, 10):
            n += mat[i, j] 
        for j in range(0, 10):
            mat[i, j] = (mat[i, j] / n) * 100
    return mat

In [76]:
#accuracy_train = getAccuracy(model, dataset, training_dataset_size)
#%store accuracy_train
%store -r accuracy_train
print("ACC on train dataset: "+str(round(accuracy_train, 4)))

ACC on train dataset: 0.9981


In [77]:
#accuracy_test = getAccuracy(model, testing, testing_dataset_size)
#%store accuracy_test
%store -r accuracy_test
print("ACC on test dataset: "+str(round(accuracy_test, 4)))

ACC on test dataset: 0.9856


In [107]:
#confusionmatrix_train = getConfusionMatrix(model, dataset)
#%store confusionmatrix_train
%store -r confusionmatrix_train
mat = normalize_matrix( confusionmatrix_train.copy() )
np.set_printoptions(suppress=True)

df = pandas.DataFrame(np.around(mat, decimals=3), columns=headers, index=headers)

In [108]:
df

,0,1,2,3,4,5,6,7,8,9
0,99.966,0.017,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000
1,0.000,99.956,0.000,0.000,0.000,0.000,0.000,0.044,0.000,0.000
2,0.017,0.017,99.916,0.000,0.000,0.000,0.000,0.034,0.000,0.017
3,0.016,0.016,0.065,99.657,0.000,0.082,0.000,0.114,0.000,0.049
4,0.034,0.086,0.017,0.000,99.743,0.000,0.000,0.017,0.000,0.103
5,0.018,0.000,0.018,0.037,0.000,99.871,0.037,0.000,0.018,0.000
6,0.017,0.101,0.000,0.000,0.000,0.017,99.814,0.000,0.051,0.000
7,0.000,0.064,0.064,0.016,0.000,0.000,0.000,99.633,0.000,0.223
8,0.051,0.034,0.000,0.000,0.000,0.000,0.000,0.000,99.880,0.034
9,0.134,0.000,0.017,0.000,0.067,0.017,0.000,0.034,0.101,99.630


In [109]:
#confusionmatrix_test = getConfusionMatrix(model, testing)
#%store confusionmatrix_test
%store -r confusionmatrix_test
mat = normalize_matrix( confusionmatrix_test.copy() )
np.set_printoptions(suppress=True)

df = pandas.DataFrame(np.around(mat, decimals=3), columns=headers, index=headers)

In [110]:
df

,0,1,2,3,4,5,6,7,8,9
0,99.490,0.102,0.000,0.000,0.204,0.000,0.204,0.000,0.000,0.000
1,0.176,99.471,0.176,0.000,0.000,0.000,0.088,0.088,0.000,0.000
2,0.097,0.291,98.450,0.000,0.000,0.000,0.097,0.581,0.484,0.000
3,0.000,0.099,0.594,97.822,0.000,0.792,0.000,0.495,0.099,0.099
4,0.102,0.102,0.102,0.000,97.658,0.000,0.509,0.000,0.000,1.527
5,0.112,0.000,0.000,0.561,0.000,98.991,0.112,0.112,0.000,0.112
6,0.731,0.313,0.313,0.000,0.000,0.104,98.225,0.000,0.313,0.000
7,0.097,0.389,0.778,0.292,0.097,0.000,0.000,98.054,0.000,0.292
8,0.103,0.103,0.205,0.000,0.000,0.000,0.000,0.205,99.179,0.205
9,0.198,0.297,0.000,0.000,0.496,0.198,0.099,0.297,0.198,98.216
